<a href="https://colab.research.google.com/github/NTT123/cute-series/blob/main/day_00_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Hello! This is the first notebook in a series of notebooks in which I learn GPU programming with Cute DSL.

[Cute DSL](https://docs.nvidia.com/cutlass/media/docs/pythonDSL/cute_dsl_general/dsl_introduction.html) is a domain-specific language (DSL) that allows us to do CUDA programming in Python directly. Cute (short for CUda TEnsor) provides a nice interface to manipulate data layouts, which is very important in GPU parallel programming. In this first notebook, we will explore how one can use Cute DSL to write very simple kernels.

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-fbaa1d9a-bea0-63e9-852f-7ccab5a89d2a)


We will use Google Colab notebooks for this series. Google Colab provides free access to T4 GPUs; all you need is a Google account. As you can see with the `nvidia-smi -L` command, we have access to a Tesla T4 GPU.

To use Cute DSL, we first need to install the `nvidia-cutlass-dsl` package. We use the ugly `sys.path.append` trick here to allow us to import `cutlass` without restarting the session.


In [2]:
%pip uninstall -y cuda-python cuda-bindings
%pip install -Uq nvidia-cutlass-dsl==4.1.0 cuda-bindings==12.9.1 cuda-python==12.9.1 svgwrite
import sys; sys.path.append("/usr/local/lib/python3.12/dist-packages/nvidia_cutlass_dsl/python_packages/")

Found existing installation: cuda-python 12.9.1
Uninstalling cuda-python-12.9.1:
  Successfully uninstalled cuda-python-12.9.1
Found existing installation: cuda-bindings 12.9.1
Uninstalling cuda-bindings-12.9.1:
  Successfully uninstalled cuda-bindings-12.9.1


## Hello World Kernel

Let's start with a hello world example:

A simple GPU program that has 3 threads, each will get its ID and print out `Hello, world!` and its name.


In [3]:
%%writefile hello_from_gpu.py
import cutlass
from cutlass import cute
import torch


@cute.kernel
def hello_kernel():
    tid, _, _ = cute.arch.thread_idx()
    cute.printf("Hello, world! I am thread #%d.", tid)


@cute.jit
def hello():
    hello_kernel().launch(
        grid=(1,1,1),
        block=(3,1,1)
    )

cutlass.cuda.initialize_cuda_context()
hello()
torch.cuda.synchronize()

Overwriting hello_from_gpu.py


In [4]:
!python hello_from_gpu.py

Hello, world! I am thread #0.
Hello, world! I am thread #1.
Hello, world! I am thread #2.


The `hello` function is decorated with `cute.jit`, which indicates this function will be compiled and run on the CPU. Meanwhile, the `hello_kernel` function is decorated with `cute.kernel`, which indicates this function will be compiled and run on the GPU.

We use the function `cute.arch.thread_idx()` to get the `threadIdx` which is a integer tupple of size 3 for three dimension x, y, z of the block. Similarly, we have the function `cute.arch.block_idx()` to get blockIdx. To get the size of a block, one can use `cute.arch.block_dim()`.

We use the syntax `hello_kernel().launch( grid=(1,1,1), block=(3,1,1) )` to launch the hello kernel, in which we specify a grid size of 1 block. And block size of 3 threads.

The `cute.printf` will be run at runtime and can be very helpful for debugging. However, `cute.printf`'s output does not show up when we run it in a notebook cell 😞, so we will not use it often in this series.


## Add Vector Kernel
Let's move on to a more practical example: adding two vectors.

Given two vectors `A` and `B` of length `N`, compute the output vector `C = A + B`.


In [5]:
import torch
from torch.testing import assert_close

import cutlass
from cutlass import cute
from cutlass.cute.runtime  import from_dlpack
from functools import partial


@cute.kernel
def add_vec_kernel(gA: cute.Tensor, gB: cute.Tensor, gC: cute.Tensor):
    tid, _, _ = cute.arch.thread_idx()
    bid, _, _ = cute.arch.block_idx()
    blk_size, _, _ = cute.arch.block_dim()
    i = tid + bid * blk_size

    gC[i] = gA[i] + gB[i]


@cute.jit
def add_vec(gA: cute.Tensor, gB: cute.Tensor, gC: cute.Tensor):
    N = cute.size(gA, [0])
    add_vec_kernel(gA, gB, gC).launch(
        grid = (N//1024, 1, 1),
        block = (1024, 1, 1)
    )


cutlass.cuda.initialize_cuda_context()
torch.manual_seed(0)
A = torch.randn(1024*512*1024, dtype=torch.float16, device='cuda')
B = torch.randn_like(A)
C = torch.empty_like(A)

A_ = from_dlpack(A, assumed_align=256)
B_ = from_dlpack(B, assumed_align=256)
C_ = from_dlpack(C, assumed_align=256)

add_vec(A_, B_, C_)
assert_close(C, A+B)

We divide the vectors into blocks, each block taking care of 1024 elements in the vectors.
For each block, we will use 1024 threads, each thread taking care of 1 element in the vectors.

A GPU is a collection of Streaming Multiprocessors (SMs) that run mostly independently from each other (you can imagine an SM as a CPU core). By dividing the problem into independent blocks, we can simply send different blocks to different SMs and allow them to be processed in parallel.
In our case, a T4 GPU has 40 SMs.

Let's do a simple benchmarking to measure the runtime and memory bandwidth of our kernel.


In [6]:
import gc

def benchmark(fn, *, num_bytes, warnup_iter, run_iter):
    start_event = [torch.cuda.Event(enable_timing=True) for _ in range(run_iter)]
    end_event = [torch.cuda.Event(enable_timing=True) for _ in range(run_iter)]
    torch.cuda.synchronize()
    for _ in range(warnup_iter):
        fn()
    for i in range(run_iter):
        A.random_()
        B.random_()
        C.random_()
        start_event[i].record()
        fn()
        end_event[i].record()
    torch.cuda.synchronize()
    ts = []
    for i in range(run_iter):
        elapsed_time = start_event[i].elapsed_time(end_event[i])
        ts.append(elapsed_time)
    avg_time = sum(ts) / len(ts)
    mem_bandwidth = num_bytes / (1e9) / (avg_time / 1e3)
    print(f"Avg time: {avg_time:.2f} ms")
    print(f"Bandwidth: {mem_bandwidth:.2f} GB/s")


# total read + write bytes
NUM_BYTES = A.nbytes * 3
compiled_kernel = cute.compile(add_vec, A_, B_, C_)
benchmark(lambda: compiled_kernel(A_, B_, C_), num_bytes=NUM_BYTES, warnup_iter=100, run_iter=100)

Avg time: 15.72 ms
Bandwidth: 204.93 GB/s


Looking at the `add_vec_kernel` kernel, we can divide it into two sub-tasks:

* **data layout**: the first four lines compute the index of the item that the current thread is working on,
* **computation**: the last line does the actual computation.

It is a common task in GPU programming to specify how the data is laid out on the GPU hardware. Cute is designed to help us with this task!

A layout is a linear map from a set of integer coordinates to a flat coordinate. A layout is defined by its shape and stride for each coordinate. The output coordinate is computed as the dot product between the input coordinates and the stride.

```python
layout = cute.make_layout(shape=(4, 8), stride=(1, 4))
```

We can use the `cute.make_layout` function to create a layout. The layout in the above example has a 2D shape of `(4, 8)` and a stride of `(1, 4)`. So, a coordinate of `(2, 5)` will be mapped to `2 x 1 + 5 x 4 = 22`.

From this, we can build a data layout for our `add-vector` kernel as follows.


In [7]:
@cute.kernel
def add_vec_kernel_w_layout(
    gA: cute.Tensor,
    gB: cute.Tensor,
    gC: cute.Tensor,
    layout: cute.Layout
):
    tid, _, _ = cute.arch.thread_idx()
    bid, _, _ = cute.arch.block_idx()
    thrA = cute.composition(gA, layout)[None, tid, bid]
    thrB = cute.composition(gB, layout)[None, tid, bid]
    thrC = cute.composition(gC, layout)[None, tid, bid]
    thrC[None] = thrA.load() + thrB.load()


@cute.jit
def add_vec_layout(mA: cute.Tensor, mB: cute.Tensor, mC: cute.Tensor):
    N = cute.size(mA, [0])
    # value-thread-block layout
    layout = cute.make_layout(
        shape=(1, 1024, N//1024),
    )

    grid_size = cute.size(layout, [2])
    block_size = cute.size(layout, [1])

    add_vec_kernel_w_layout(mA, mB, mC, layout).launch(
        grid = (grid_size, 1, 1),
        block = (block_size, 1, 1)
    )

C.zero_()
add_vec_layout(A_, B_, C_)
assert_close(C, A+B)

compiled_kernel = cute.compile(add_vec_layout, A_, B_, C_)
benchmark(lambda: compiled_kernel(A_, B_, C_), num_bytes=NUM_BYTES, warnup_iter=20, run_iter=100)

Avg time: 15.71 ms
Bandwidth: 205.07 GB/s


We created a value-thread-block layout for our data that allows us to locate the data using `block_idx` and `thread_idx`. Note that when the stride is not specified, it is computed automatically such that the first coordinate has stride 1. This is different from `torch.Tensor`’s default stride, where the last coordinate has stride 1.

A `cute.Tensor` is simply a pair of a data source and a layout. This allows us to compute the composition of a tensor and a layout. For example, `cute.composition(gA, layout)[None, None, bid]` will return a new tensor that has a layout mapping `(value_idx, thread_idx, block_idx)` to the physical address (actually, the offset) of a data item in `gA`.

In summary, `cute.composition(gA, layout)[None, tid, bid]` return a slice of `gA` that belongs to the current thread.

## Faster Add Vector Kernel

In the above kernel, each thread loads 1 data item of size 2 bytes (float16) from global memory to a register. A warp of 32 threads will load a total of 64 contiguous bytes from global memory. However, this does not saturate the memory bandwidth, as the T4 GPU can load a total of 128 bytes at once. To fully saturate the memory bandwidth, each thread should load 2 items (4 bytes) at once. We can easily do that by modifying our layout. The rest of the kernel needs no change!


In [8]:
@cute.kernel
def faster_add_vec_kernel_w_layout(
    gA: cute.Tensor,
    gB: cute.Tensor,
    gC: cute.Tensor,
    btv_layout: cute.Layout
):
    tid, _, _ = cute.arch.thread_idx()
    bid, _, _ = cute.arch.block_idx()
    thrA = cute.composition(gA, btv_layout)[None, tid, bid]
    thrB = cute.composition(gB, btv_layout)[None, tid, bid]
    thrC = cute.composition(gC, btv_layout)[None, tid, bid]
    thrC[None] = thrA.load() + thrB.load()


@cute.jit
def faster_add_vec_layout(mA: cute.Tensor, mB: cute.Tensor, mC: cute.Tensor):
    N = cute.size(mA, [0])
    # value-thread-block layout
    btv_layout = cute.make_layout(
        shape=(2, 512, N//1024),
    )

    grid_size = cute.size(btv_layout, [2])
    block_size = cute.size(btv_layout, [1])

    faster_add_vec_kernel_w_layout(mA, mB, mC, btv_layout).launch(
        grid = (grid_size, 1, 1),
        block = (block_size, 1, 1)
    )

C.zero_()
add_vec_layout(A_, B_, C_)
assert_close(C, A+B)

compiled_kernel = cute.compile(faster_add_vec_layout, A_, B_, C_)
benchmark(lambda: compiled_kernel(A_, B_, C_), num_bytes=NUM_BYTES, warnup_iter=20, run_iter=100)

Avg time: 12.22 ms
Bandwidth: 263.69 GB/s


## Profiling

Finally, let's profile the kernel with `ncu` to check if our kernel is in good shape.


In [9]:
%%writefile add_vec_kernel.py
import torch
from torch.testing import assert_close

import cutlass
from cutlass import cute
from cutlass.cute.runtime  import from_dlpack
from functools import partial


@cute.kernel
def add_vec_kernel(gA: cute.Tensor, gB: cute.Tensor, gC: cute.Tensor):
    tid, _, _ = cute.arch.thread_idx()
    bid, _, _ = cute.arch.block_idx()
    blk_size, _, _ = cute.arch.block_dim()
    i = tid + bid * blk_size

    gC[i] = gA[i] + gB[i]


@cute.jit
def add_vec(gA: cute.Tensor, gB: cute.Tensor, gC: cute.Tensor):
    N = cute.size(gA, [0])
    add_vec_kernel(gA, gB, gC).launch(
        grid = (N//1024, 1, 1),
        block = (1024, 1, 1)
    )


cutlass.cuda.initialize_cuda_context()
torch.manual_seed(0)
A = torch.randn(1024*512*1024, dtype=torch.float16, device='cuda')
B = torch.randn_like(A)
C = torch.empty_like(A)

A_ = from_dlpack(A, assumed_align=256)
B_ = from_dlpack(B, assumed_align=256)
C_ = from_dlpack(C, assumed_align=256)

add_vec(A_, B_, C_)
assert_close(C, A+B)


@cute.kernel
def faster_add_vec_kernel_w_layout(
    gA: cute.Tensor,
    gB: cute.Tensor,
    gC: cute.Tensor,
    btv_layout: cute.Layout
):
    tid, _, _ = cute.arch.thread_idx()
    bid, _, _ = cute.arch.block_idx()
    thrA = cute.composition(gA, btv_layout)[None, tid, bid]
    thrB = cute.composition(gB, btv_layout)[None, tid, bid]
    thrC = cute.composition(gC, btv_layout)[None, tid, bid]
    thrC[None] = thrA.load() + thrB.load()


@cute.jit
def faster_add_vec_layout(mA: cute.Tensor, mB: cute.Tensor, mC: cute.Tensor):
    N = cute.size(mA, [0])
    # value-thread-block layout
    btv_layout = cute.make_layout(
        shape=(2, 512, N//1024),
    )

    grid_size = cute.size(btv_layout, [2])
    block_size = cute.size(btv_layout, [1])

    faster_add_vec_kernel_w_layout(mA, mB, mC, btv_layout).launch(
        grid = (grid_size, 1, 1),
        block = (block_size, 1, 1)
    )

C.zero_()
faster_add_vec_layout(A_, B_, C_)
assert_close(C, A+B)

Overwriting add_vec_kernel.py


In [10]:
del A, B, C, A_, B_, C_
torch.cuda.empty_cache()

In [11]:
!ncu -k 'regex:kernel_cutlass_.+' python add_vec_kernel.py

==PROF== Connected to process 4512 (/usr/bin/python3.12)
==PROF== Profiling "kernel_cutlass_add_vec_kernel_tensorptrf16gmemalign256o5368709121_tensorptrf16gmemalign256o5368709121_tensorptrf16gmemalign256o5368709121_0": 0%....50%....100% - 9 passes
==PROF== Profiling "kernel_cutlass_faster_add_vec_kernel_w_layout_tensorptrf16gmemalign256o5368709121_tensorptrf16gmemalign256o5368709121_tensorptrf16gmemalign256o5368709121_0": 0%
==WARNING== Backing up device memory in system memory. Kernel replay might be slow. Consider using "--replay-mode application" to avoid memory save-and-restore.
....50%....100% - 9 passes
==PROF== Disconnected from process 4512
[4512] python3.12@127.0.0.1
  kernel_cutlass_add_vec_kernel_tensorptrf16gmemalign256o5368709121_tensorptrf16gmemalign256o5368709121_tensorptrf16gmemalign256o5368709121_0 (524288, 1, 1)x(1024, 1, 1), Context 2, Stream 14, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metri

As expected, the original kernel (`kernel_cutlass_add_vec_kernel`) only achieved a peak memory throughput of 60%, while our best kernel (`kernel_cutlass_faster_add_vec_kernel_w_layout`) achieved a peak memory throughput of 90% 😀.

---

THE END
